# **Hello Credit 0.0.3 (Model Demo)**


# NOTES:

* Returns are more important than the methodology
* Backtest against Moody's, Bloomberg Models
* "Do not let, perfect stand in the way of good"


In [12]:
import joblib
import numpy as np
import pandas as pd

In [2]:
import pandas as pd

# Initial data
data = {
    "Company": ["Company A", "Company B", "Company C", "Company D", "Company E", "Company F"],
    "Return_on_Assets": [0.02, 0.04, 0.07, 0.03, 0.05, 0.09],
    "Operating_Margin_Ratio": [0.03, 0.10, 0.20, 0.05, 0.12, 0.25],
    "Net_Profit_Margin": [0.02, 0.06, 0.12, 0.03, 0.07, 0.15],
    "Debt_to_EBITDA": [5.0, 3.5, 1.5, 4.0, 2.5, 1.0],
    "EBITDA_to_Interest_Expense": [1.2, 2.0, 3.5, 1.5, 2.5, 4.0],
    "Asset_Turnover": [0.4, 0.8, 1.6, 0.5, 1.0, 2.0]
}
df = pd.DataFrame(data)

In [4]:
df = pd.read_excel("metrics.xlsx")

In [13]:
df = df.rename(columns={'Unnamed: 0':'Company'})

In [15]:
classification_ranges = joblib.load("metrics.joblib")

In [34]:
classification_ranges

{'tot_debt_to_tot_eqy': {1: (-inf, 50.647210600805735),
  0: (50.647210600805735, 65.03290102599539),
  -1: (65.03290102599539, inf)},
 'interest_coverage_ratio': {1: (8.858455386626364, inf),
  0: (5.633747012489427, 8.858455386626364),
  -1: (-inf, 5.633747012489427)},
 'return_on_asset': {1: (9.589846015144378, inf),
  0: (5.854837291308153, 9.589846015144378),
  -1: (-inf, 5.854837291308153)},
 'tot_debt_to_ebitda': {1: (-inf, 1.060896398933337),
  0: (1.060896398933337, 1.3367408348954506),
  -1: (1.3367408348954506, inf)},
 'ebitda_to_tot_int_exp': {1: (13.174393594464583, inf),
  0: (8.484247144183694, 13.174393594464583),
  -1: (-inf, 8.484247144183694)}}

In [18]:
# Classification values and ranges
classification_values = {-1: 1, 0: 2, 1: 3}

# classification_ranges = {
#     'Return_on_Assets': {'Poor': (None, 0.02), 'Fair': (0.02, 0.05), 'Good': (0.05, None)},
#     'Operating_Margin_Ratio': {'Poor': (None, 0.10), 'Fair': (0.10, 0.20), 'Good': (0.20, None)},
#     'Net_Profit_Margin': {'Poor': (None, 0.05), 'Fair': (0.05, 0.10), 'Good': (0.10, None)},
#     'Debt_to_EBITDA': {'Good': (None, 2.0), 'Fair': (2.0, 4.0), 'Poor': (4.0, None)},  # Note: Inverted logic for Debt to EBITDA
#     'EBITDA_to_Interest_Expense': {'Poor': (None, 2.0), 'Fair': (2.0, 4.0), 'Good': (4.0, None)},
#     'Asset_Turnover': {'Poor': (None, 0.5), 'Fair': (0.5, 1.5), 'Good': (1.5, None)}
# }

# Function to classify metric values into "Poor", "Fair", and "Good"
def classify_metric(value, metric):
    ranges = classification_ranges[metric]
    for classification, (low, high) in ranges.items():
        if low is None and value <= high:
            return classification
        elif high is None and value > low:
            return classification
        elif low is not None and high is not None and low <= value <= high:
            return classification
    return "Unknown"

# Apply classification for each metric
for metric in classification_ranges.keys():
    df[f'{metric}_Class'] = df[metric].apply(lambda x: classify_metric(x, metric))

# Function to calculate overall score and classification
def calculate_and_classify_overall(row):
    scores = [classification_values[row[f'{metric}_Class']] for metric in classification_ranges]
    overall_score = sum(scores) / len(scores)
    overall_class = 1 if overall_score > 2.5 else 0 if overall_score > 1.5 else -1
    return pd.Series([overall_score, overall_class], index=['Overall_Score', 'Overall_Class'])

# Calculate overall scores and classifications
df[['Overall_Score', 'Overall_Class']] = df.apply(calculate_and_classify_overall, axis=1)

In [19]:
df.set_index('Company', inplace=True)

In [27]:
df = df.replace(-1, "Poor").replace(0, "Fair").replace(1, "Good")

## Actual Metrics (Financial Ratios)

In [29]:
df.iloc[: , :5]

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
Company,,,,,
BLU SJ Equity,32.809328,6.346575,4.215240,1.150158,7.547418
MTN SJ Equity,65.116785,8.547827,7.405181,1.305329,13.114130
TKG SJ Equity,56.593172,3.495262,6.401370,1.430977,8.796523
VOD SJ Equity,65.004940,9.790341,16.143841,0.793112,13.355185


## Metric Classifications

In [33]:
df.iloc[: , 5:10]

,tot_debt_to_tot_eqy_Class,interest_coverage_ratio_Class,return_on_asset_Class,tot_debt_to_ebitda_Class,ebitda_to_tot_int_exp_Class
Company,,,,,
BLU SJ Equity,Good,Fair,Poor,Fair,Poor
MTN SJ Equity,Poor,Fair,Fair,Fair,Fair
TKG SJ Equity,Fair,Poor,Fair,Poor,Fair
VOD SJ Equity,Fair,Good,Good,Good,Good


## Overall Scores and Classifications

In [37]:
df.iloc[:, -2:]

,Overall_Score,Overall_Class
Company,,
BLU SJ Equity,1.8,Fair
MTN SJ Equity,1.8,Fair
TKG SJ Equity,1.6,Fair
VOD SJ Equity,2.8,Good
